In [3]:
import numpy as np
import json

res_path = "../results/pearson/front/aggregate_50/res_dummy.json"

with open(res_path, "r") as jsonfile:
    res = json.load(jsonfile)

tx = res['TX']
rx = res['RX']

count = 0
c_rep = 0
for (s_id, data) in tx.items():
    c = data['count']
    c_rep += c
    print(f'{s_id}: {c} reps')
    count += 1

ratio_samples = count/315.0
samp_succ = c_rep/(count*20.0)
overall_succ = c_rep/(315.0*20.0)
print(f'\ntotal samples: {count} ({ratio_samples*100:.2f}%)')
print(f'total successful results: {c_rep} ({samp_succ*100:.2f}%) ({overall_succ*100:.2f}%)')

127: 20 reps
154: 20 reps
246: 20 reps
094: 20 reps
033: 20 reps
141: 20 reps
032: 19 reps
201: 20 reps
072: 20 reps
098: 20 reps
064: 20 reps
229: 20 reps
012: 20 reps
102: 20 reps
017: 20 reps
225: 20 reps
066: 20 reps
029: 20 reps
097: 20 reps
071: 19 reps
302: 0 reps
206: 20 reps
014: 20 reps
067: 20 reps
155: 20 reps
024: 20 reps
147: 5 reps
062: 20 reps
133: 20 reps
020: 20 reps
157: 20 reps
119: 20 reps
055: 20 reps
008: 15 reps
003: 20 reps
006: 20 reps
007: 18 reps
177: 20 reps
054: 20 reps
039: 19 reps
002: 20 reps
153: 20 reps
021: 19 reps
270: 20 reps
107: 20 reps
250: 12 reps
111: 20 reps
015: 2 reps
074: 20 reps
260: 20 reps
088: 20 reps
124: 20 reps
099: 20 reps
150: 20 reps
286: 20 reps
151: 20 reps
120: 20 reps
087: 20 reps
075: 20 reps
034: 20 reps
026: 20 reps
285: 2 reps
200: 2 reps
004: 20 reps
307: 20 reps
036: 20 reps
312: 20 reps
090: 6 reps
057: 20 reps
129: 20 reps
096: 20 reps
236: 20 reps
025: 20 reps
038: 20 reps
093: 20 reps
009: 20 reps
100: 20 reps
148: 